# week 6

In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

# load dataset
df = pd.read_csv('homework_6.1.csv')

# split treated and untreated groups
treated = df[df['X'] == 1].reset_index(drop=True)
untreated = df[df['X'] == 0].reset_index(drop=True)

# confounder Z for matching
Z_treated = treated[['Z']]
Z_untreated = untreated[['Z']]

# nearest neighbor models
nn_treat = NearestNeighbors(n_neighbors=1).fit(Z_treated)
nn_untreat = NearestNeighbors(n_neighbors=1).fit(Z_untreated)

# match each treated with nearest untreated and vice versa
treated_to_untreated_idx = nn_untreat.kneighbors(Z_treated, return_distance=False).flatten()
untreated_to_treated_idx = nn_treat.kneighbors(Z_untreated, return_distance=False).flatten()

# counterfactual matches
treated_counterfactual = untreated.iloc[treated_to_untreated_idx]
untreated_counterfactual = treated.iloc[untreated_to_treated_idx]

# individual treatment effects
ite_treated = treated['Y'].values - treated_counterfactual['Y'].values
ite_untreated = untreated_counterfactual['Y'].values - untreated['Y'].values

# ATE
ATE = np.mean(np.concatenate([ite_treated, ite_untreated]))

# ATT
ATT = np.mean(ite_treated)

# ATU
ATU = np.mean(ite_untreated)

# MTE — maximum among untreated effects
MTE = np.max(ite_untreated)

ATE, ATT, ATU, MTE

(np.float64(1.6952701427487222),
 np.float64(1.846408507179443),
 np.float64(1.5494765534845105),
 np.float64(2.1724698851117035))